In [15]:
import pandas as pd
import json
import requests
from config import AlphaVan_key

In [3]:
url = f'https://www.alphavantage.co/query?function=COFFEE&interval=monthly&apikey={AlphaVan_key}'
r = requests.get(url)
data = r.json()


{
    "data": [
        {
            "date": "2023-08-01",
            "value": "186.346956521739"
        },
        {
            "date": "2023-07-01",
            "value": "193.494285714286"
        },
        {
            "date": "2023-06-01",
            "value": "207.385"
        },
        {
            "date": "2023-05-01",
            "value": "220.121304347826"
        },
        {
            "date": "2023-04-01",
            "value": "229.9555"
        },
        {
            "date": "2023-03-01",
            "value": "222.663913043478"
        },
        {
            "date": "2023-02-01",
            "value": "229.506"
        },
        {
            "date": "2023-01-01",
            "value": "206.76"
        },
        {
            "date": "2022-12-01",
            "value": "210.388636363636"
        },
        {
            "date": "2022-11-01",
            "value": "213.718181818182"
        },
        {
            "date": "2022-10-01",
            "value": "240.

In [41]:
df = pd.DataFrame(data["data"])
filtered_df = df[df["value"] != "."]
filtered_df["value"] = filtered_df["value"].astype(float).round(2)
filtered_df

C:\Users\Joey\AppData\Local\Temp\ipykernel_10228\475285374.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["value"] = filtered_df["value"].astype(float).round(2)


,date,value
0,2023-08-01,186.35
1,2023-07-01,193.49
2,2023-06-01,207.38
3,2023-05-01,220.12
4,2023-04-01,229.96
...,...,...
399,1990-05-01,92.02
400,1990-04-01,93.73
401,1990-03-01,93.96
402,1990-02-01,84.01
